从 SQLite 提取文本数据

In [11]:
import sqlite3
import pandas as pd

# 连接数据库
conn = sqlite3.connect('CBDB.db')
cursor = conn.cursor()
df = pd.read_sql_query("SELECT * FROM ADDRESSES", conn)
conn.close()

texts = df['c_name_chn'].tolist()
metadata = df[['c_name', 'c_addr_id']].to_dict('records')

print(texts[:100])
print(metadata[:100])

['寶坻', '新泰', '宛平', '平晉', '堅州', '漷州', '牟平', '德平', '太谷', '臨朐', '安德', '平原', '忻州', '齊河', '清平', '鉅鹿', '天成', '臨淄', '王屋', '濬州', '滕州', '壽光', '磁州', '兗州', '霸州', '博興州', '文登', '安陽', '威州', '高苑', '膠州', '益都', '慶都', '永年', '順州', '榆次', '撫寧', '河陽', '清源', '固安州', '東安州', '龍慶州', '岳陽', '交城', '夏津', '香河等處巡檢司', '棣州', '汾州', '濰州', '和眾', '檀州', '壽陽', '開平', '元氏', '潞州', '寶昌州', '絳州', '威寧', '萊蕪', '濟源', '廣平', '永清', '昌平', '大興', '徐溝', '通州', '涿州', '高原', '清河', '鉅野', '虞城', '大名', '汲縣', '魏縣', '開州', '濟州', '林州', '陵州', '臨漳', '遷安', '元城', '鄒平', '樂安', '內邱', '滑州', '欒城', '邱縣', '東寧路', '靈壽', '中和', '平山', '趙州', '曲周', '真定', '唐山', '懷安', '土山', '南樂', '濟陽', '輝州']
[{'c_name': 'Baodi', 'c_addr_id': 16819}, {'c_name': 'Xintai', 'c_addr_id': 16799}, {'c_name': 'Wanping', 'c_addr_id': 16816}, {'c_name': 'Pingjin', 'c_addr_id': 17194}, {'c_name': 'Jian Zhou', 'c_addr_id': 17230}, {'c_name': 'Huo Zhou', 'c_addr_id': 16838}, {'c_name': 'Mouping', 'c_addr_id': 16811}, {'c_name': 'Deping', 'c_addr_id': 16805}, {'c_name': 'Taigu', 'c_addr_id': 17197}, {'c

生成文本嵌入

In [12]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('BAAI/bge-small-zh-v1.5')
embeddings = model.encode(texts, show_progress_bar=True)

KeyboardInterrupt: 

构建 FAISS 向量索引并保存

In [6]:
import faiss
import numpy as np
import pickle

embeddings = np.array(embeddings).astype('float32')

faiss.normalize_L2(embeddings)

# 创建使用内积的索引（归一化后，内积 = 余弦相似度）
d = embeddings.shape[1]
index = faiss.IndexFlatIP(d)

# 添加向量
index.add(embeddings)
print(f"✅ 已添加 {index.ntotal} 条归一化向量")

# 保存索引和数据
faiss.write_index(index, 'CBDB.index')

with open('E:/LLM_CBDB/texts.pkl', 'wb') as f:
    pickle.dump(texts, f)

with open('E:/LLM_CBDB/metadata.pkl', 'wb') as f:
    pickle.dump(metadata, f)

print("💾 FAISS 余弦相似度索引和数据已保存！")

✅ 已添加 51267 条归一化向量
💾 FAISS 余弦相似度索引和数据已保存！


In [7]:
data_to_save = {
    'texts': texts,
    'metadatas': metadata
}
with open('E:/LLM_CBDB/faiss_data.pkl', 'wb') as f:
    pickle.dump(data_to_save, f)

RAG检索

In [8]:
import faiss
import numpy as np
import pickle
from sentence_transformers import SentenceTransformer

class FAISSRetriever:
    def __init__(self, index_path = 'CBDB.index', data_path = "faiss_data.pkl", model_name = 'BAAI/bge-small-zh-v1.5'):
        self.index = faiss.read_index(index_path)
        with open(data_path, 'rb') as f:
            data = pickle.load(f)
            self.texts = data['texts']
            self.metadatas = data['metadatas']
        self.model = SentenceTransformer(model_name)

    def search(self, query, top_k=3):
        query_vec = self.model.encode([query], normalize_embeddings=True).astype('float32')
        distances, indices = self.index.search(query_vec, top_k)

        results = []

        for i, idx in enumerate(indices[0]):
            if idx == -1:
                continue
            results.append({
                'text': self.texts[idx],
                'metadata': self.metadatas[idx],
                'score': distances[0][i]
            })

        return  results

调用LLM

In [9]:
# llm.py —— 使用本地 Qwen2.5:7B 生成答案
import requests
import json

def generate_answer_with_context(query, retrieved_docs, model="qwen2.5:7b"):
    """
    使用本地 Qwen2.5:7B 模型生成答案
    """
    # 构建上下文
    context = "\n\n".join([
        f"【相关文档{i+1}】（标题：{doc['metadata'].get('title', '无标题')}）\n{doc['text']}"
        for i, doc in enumerate(retrieved_docs)
    ])

    # 构建 Prompt（适配 Qwen 的对话格式）
    prompt = f"""<|im_start|>system
你是一个智能助手，请根据提供的上下文信息，准确、简洁地回答用户的问题。如果上下文没有提供足够信息，请回答“根据现有资料无法回答”。<|im_end|>
<|im_start|>user
上下文：
{context}

问题：{query}<|im_end|>
<|im_start|>assistant
"""

    try:
        # 调用 Ollama API
        response = requests.post(
            "http://localhost:11434/api/generate",
            json={
                "model": model,
                "prompt": prompt,
                "stream": False,
                "options": {
                    "temperature": 0.2,
                    "num_ctx": 4096,  # 上下文长度
                    "top_p": 0.9,
                    "repeat_penalty": 1.1
                }
            },
            timeout=120  # 大模型生成可能较慢，设置超时
        )

        if response.status_code == 200:
            result = response.json()
            return result.get("response", "模型未返回有效内容").strip()
        else:
            return f"❌ 模型调用失败：HTTP {response.status_code}"

    except Exception as e:
        return f"❌ 本地模型调用异常：{str(e)}"

In [17]:
retriever = FAISSRetriever(
        index_path="CBDB.index",
        data_path="faiss_data.pkl",
        model_name='BAAI/bge-small-zh-v1.5'  # 如果是中文数据，建议用中文嵌入模型
    )

query = "c_name_chn当中龍驤衛出现了多少次？"
print(f"❓ 用户问题：{query}\n")

# 检索
results = retriever.search(query, top_k=3)

print("📄 检索到的相关文档：")
for i, res in enumerate(results):
    score = res['score']
    title = res['metadata'].get('title', '无标题')
    preview = res['text'][:150].replace('\n', ' ')
    print(f"【{i+1}】相似度：{score:.4f} | 标题：{title}")
    print(f"    内容：{preview}...\n")

# 生成答案（非流式）
print("🤔 正在调用 Qwen2.5-7B 生成答案...\n")
answer = generate_answer_with_context(query, results, model="qwen2.5:7b")
print("✅ AI 生成答案：")
print(answer)

❓ 用户问题：c_name_chn当中龍驤衛出现了多少次？

📄 检索到的相关文档：
【1】相似度：0.6421 | 标题：无标题
    内容：龍驤衛...

【2】相似度：0.6421 | 标题：无标题
    内容：龍驤衛...

【3】相似度：0.5825 | 标题：无标题
    内容：龍虎衛...

🤔 正在调用 Qwen2.5-7B 生成答案...

✅ AI 生成答案：
根据现有资料无法回答。因为提供的上下文信息中没有具体的文本内容或数据来统计"c_name_chn"中“龍驤衛”出现了多少次。
